Терминалогия, используемая в задании:
* обучающая выборка - выборка, которая передается в метод fit / train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.

In [28]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import missingno as msno
import lightgbm as lgb
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
from scipy.stats import probplot, ks_2samp
import warnings
from tqdm import tqdm
from typing import List, Tuple

from scipy.stats import ttest_rel

from sklearn.metrics import r2_score, roc_auc_score

from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
warnings.simplefilter("ignore")
%matplotlib inline

In [51]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [33]:
train= pd.read_csv('train.csv')
l_board = pd.read_csv('test.csv')

In [34]:
train.set_index('TransactionID',inplace=True)
l_board.set_index('TransactionID',inplace=True)

In [35]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180000 entries, 2987000 to 3166999
Columns: 393 entries, isFraud to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 541.1+ MB


In [37]:
l_board.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100001 entries, 3287000 to 3387000
Columns: 393 entries, isFraud to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 300.6+ MB


Задание 1: сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [38]:
object_features = train.select_dtypes(include=[np.object]).columns

In [39]:
train[object_features]= train[object_features].astype('category')
l_board[object_features]= l_board[object_features].astype('category')

In [62]:
x_train,x_valid,y_train,y_valid = train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.3 ,random_state=15)

In [63]:
dtrain = lgb.Dataset(data=x_train, label=y_train,categorical_feature=object_features.to_list())
dvalid = lgb.Dataset(data=x_valid, label=y_valid,categorical_feature=object_features.to_list())

In [85]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "n_estimators": 20000,
    "n_jobs": 15,
    "seed": 5
}

In [65]:
model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain, dvalid],
    categorical_feature=object_features.to_list(),
    early_stopping_rounds=90,
    verbose_eval=None
)

In [66]:
model.best_score, model.best_iteration

(defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9999718098871575)]),
              'valid_1': OrderedDict([('auc', 0.9564815499099788)])}),
 1493)

Качество модели на валидационной выборке 0,957  что кажеться очень хорошим результатом. При  этон на трайн почти единица. 
Что говорит, что модель сильно подстроилась по данные, но при этом разность не сильно большая меньеше 5%

Оценим качество на лидер борде

In [67]:
roc_auc_score(l_board['isFraud'], model.predict(l_board.drop('isFraud',axis=1)))

0.8496501937123049

Наблюдаем сильное отличие показателя качества на валидации и лидерборде, почти 10%

Задание 2: сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [68]:
x_valid,x_test,y_valid,y_test = train_test_split(x_valid,y_valid, test_size = 0.4 ,random_state=15)

In [69]:
x_valid.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32400 entries, 3083768 to 3127011
Columns: 392 entries, TransactionDT to V339
dtypes: category(14), float64(376), int64(2)
memory usage: 94.1 MB


In [70]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21600 entries, 3122360 to 3071062
Columns: 392 entries, TransactionDT to V339
dtypes: category(14), float64(376), int64(2)
memory usage: 62.8 MB


In [71]:
# Скор на валидационной выборке
roc_auc_score(y_valid, model.predict(x_valid))

0.9542313280399443

In [72]:
# Скор на тестовой выборке
roc_auc_score(y_test, model.predict(x_test))

0.9597253383379328

Как видно показатель качества не сильно отличается на валидационной и на тестовой выборке и их сильно преврсходит лидерборд. То есть модель на своих данных стабильна. Но как только появляются данне м ЛБ, все идет не так. Таким образом,  модель на ЛБ будет работать по доугому, видимо данные сильно отличаются

Задание 3: построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

-----
Вот здесь я запутался. Для данного задания нужно обучать модель на полных данных или только на разделенных(x_train , за вычетом x_valid)).

Сначало все сделал на x_train, потом посмотрел лекцию, там модель обучена на полном объеме данных. Переделал. И бустрап валидацию, так тже как и в задании делдал на тестовой выборке (самой маленькой)

In [94]:
dtrain = lgb.Dataset(data=train.drop('isFraud',axis=1), label=train['isFraud'],categorical_feature=object_features.to_list())
dvalid = lgb.Dataset(data=x_valid, label=y_valid,categorical_feature=object_features.to_list())

In [95]:
model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain,dvalid],
    categorical_feature=object_features.to_list(),
    early_stopping_rounds=90,
    verbose_eval=None
)

In [96]:
model.best_score, model.best_iteration

(defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9999987941488887)]),
              'valid_1': OrderedDict([('auc', 0.9999997936997556)])}),
 2706)

In [97]:
# доверительный интервал на данных из п.2 на основе бутстреп выборок
np.random.seed(15)
scores = create_bootstrap_metrics(y_test, model.predict(x_test), roc_auc_score)

calculate_confidence_interval(scores)

(0.9999835498445154, 1.0)

In [98]:
# Скор на лидерборде
roc_auc_score(l_board['isFraud'], model.predict(l_board.drop('isFraud',axis=1)))

0.8323428374893191

вторым способом, не знаю как правильно

In [91]:
dtrain = lgb.Dataset(data=x_train, label=y_train,categorical_feature=object_features.to_list())
dvalid = lgb.Dataset(data=x_valid, label=y_valid,categorical_feature=object_features.to_list())

In [92]:
model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain,dvalid],
    categorical_feature=object_features.to_list(),
    early_stopping_rounds=90,
    verbose_eval=None
)
model.best_score, model.best_iteration

(defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9999869003184734)]),
              'valid_1': OrderedDict([('auc', 0.954349331779755)])}),
 1615)

In [93]:
# Скор на лидерборде
roc_auc_score(l_board['isFraud'], model.predict(l_board.drop('isFraud',axis=1)))

0.8491836579831299

 Видимо данные сильно отличаются в моделе и лидерборде.

Задание 4: выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

Здесь я то же не очень понимаю задаание.
Adversarial Validation между x_train и  x_valid?, так сделано в лекции , и в задании указано 

 Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. Для других целей использовать выборку запрещено!.
    
    Но я не понимаю для чего так делать, это ничего не даст.
    Нужно делать с данными которые для лидерборда, что бы понять как они отличаются.
    Поскольу рабочих данных все прекрасно и так


In [99]:
x_adv = pd.concat([
    train.drop('isFraud',axis=1), l_board.drop('isFraud',axis=1)], axis=0
)
y_adv = np.hstack((np.zeros(train.shape[0]), np.ones(l_board.shape[0])))
assert x_adv.shape[0] == y_adv.shape[0]

In [108]:
x_adv[object_features]= x_adv[object_features].astype('category')

In [109]:
x_adv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280001 entries, 2987000 to 3387000
Columns: 392 entries, TransactionDT to V339
dtypes: category(14), float64(376), int64(2)
memory usage: 813.4 MB


In [113]:
np.unique(y_adv),len(y_adv)

(array([0., 1.]), 280001)

In [295]:
dtrain = lgb.Dataset(data=x_adv, label=y_adv,categorical_feature=object_features.to_list())

In [ ]:
model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain],
    categorical_feature=object_features.to_list(),
    early_stopping_rounds=90,
    verbose_eval=None
)
model.best_score, model.best_iteration

In [264]:
a = list(zip(model.feature_name(), model.feature_importance()))

In [269]:
a.sort(key=lambda x: x[1],reverse=True)
a

[('C13', 9),
 ('C1', 4),
 ('C14', 4),
 ('D2', 4),
 ('V87', 4),
 ('C12', 3),
 ('V156', 3),
 ('V317', 3),
 ('card2', 3),
 ('C2', 2),
 ('C8', 2),
 ('M4', 2),
 ('TransactionAmt', 2),
 ('V44', 2),
 ('V53', 2),
 ('V67', 2),
 ('card1', 2),
 ('card3', 2),
 ('C10', 1),
 ('C11', 1),
 ('C5', 1),
 ('C6', 1),
 ('C9', 1),
 ('D1', 1),
 ('D15', 1),
 ('D9', 1),
 ('TransactionDT', 1),
 ('V105', 1),
 ('V12', 1),
 ('V143', 1),
 ('V155', 1),
 ('V170', 1),
 ('V173', 1),
 ('V189', 1),
 ('V201', 1),
 ('V206', 1),
 ('V239', 1),
 ('V243', 1),
 ('V244', 1),
 ('V261', 1),
 ('V268', 1),
 ('V283', 1),
 ('V294', 1),
 ('V313', 1),
 ('V323', 1),
 ('V45', 1),
 ('V58', 1),
 ('V62', 1),
 ('V63', 1),
 ('V77', 1),
 ('addr1', 1),
 ('addr2', 1),
 ('card5', 1),
 ('C3', 0),
 ('C4', 0),
 ('C7', 0),
 ('D10', 0),
 ('D11', 0),
 ('D12', 0),
 ('D13', 0),
 ('D14', 0),
 ('D3', 0),
 ('D4', 0),
 ('D5', 0),
 ('D6', 0),
 ('D7', 0),
 ('D8', 0),
 ('M1', 0),
 ('M2', 0),
 ('M3', 0),
 ('M5', 0),
 ('M6', 0),
 ('M7', 0),
 ('M8', 0),
 ('M9', 0),


In [117]:
pred = model.predict(x_adv)

In [119]:
pd.cut(
    pred, bins=np.arange(0, 1.01, 0.1)
).value_counts().sort_index()

(0.0, 0.1]    179827
(0.1, 0.2]       100
(0.2, 0.3]        43
(0.3, 0.4]        19
(0.4, 0.5]        10
(0.5, 0.6]         2
(0.6, 0.7]         5
(0.7, 0.8]        20
(0.8, 0.9]        93
(0.9, 1.0]     99882
dtype: int64

In [120]:
len(train),len(l_board)

(180000, 100001)

In [121]:
pd.cut(
    pred, bins=np.arange(0, 1.01, 0.5)
).value_counts().sort_index()

(0.0, 0.5]    179999
(0.5, 1.0]    100002
dtype: int64

Модель соврешенно точно, практически на 100 % отличает какие данные из какой выборки, данные очень разные. Строить валидацию из данных для обучения с приличной точностью - не получиться

In [122]:
predict = np.where(pred>=0.5,1,0)

In [126]:
sum(predict !=y_adv)

1

Только одна ошибка из 280000

Даже если взять строки от 0,1 до 0,9 их не будет даже две сотни. Мало что бы строить валидацию и не сильно похоже. 

Проверим

In [270]:
# индексы, где значение вероятности боле0,1
ind_val = np.where((y_adv ==0)&(pred>0.1))[0]

In [271]:
ind_val.shape

(173,)

In [275]:
X_test = train.iloc[ind_val]

In [276]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 3166742 to 3166998
Columns: 393 entries, isFraud to V339
dtypes: category(14), float64(376), int64(3)
memory usage: 523.3 KB


In [277]:
X_train= train.drop(X_test.index)

In [278]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179827 entries, 2987000 to 3166999
Columns: 393 entries, isFraud to V339
dtypes: category(14), float64(376), int64(3)
memory usage: 523.8 MB


In [279]:
y_train=X_train['isFraud']
y_test=X_test['isFraud']
X_train=X_train.drop('isFraud',axis=1)
X_test=X_test.drop('isFraud',axis=1)


In [280]:
dtrain = lgb.Dataset(data=X_train, label=y_train,categorical_feature=object_features.to_list())
dvalid = lgb.Dataset(data=X_test, label=y_test,categorical_feature=object_features.to_list())

In [281]:
model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain,dvalid],
    categorical_feature=object_features.to_list(),
    early_stopping_rounds=90,
    verbose_eval=None
)
model.best_score, model.best_iteration

(defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9611678778725501)]),
              'valid_1': OrderedDict([('auc', 0.9985207100591716)])}),
 135)

In [282]:
# Скор на лидерборде
roc_auc_score(l_board['isFraud'], model.predict(l_board.drop('isFraud',axis=1)))

0.8720878429382346

Совсем не понятная мне ситуация. Мы убрали на валидацию, строки более похожие на тест. Сама модель, стала более сильнее отличаться от лидер борда.
По идее на валидации должен ухудшиться показатель и приблизиться к лидерборду, но все наоборот. И как не странно модель на лидерборде дает большый результат

Попробуем убрать столюбец С13, который сильно влияет на разность данных и посомтрим, что будет в целом

In [289]:
dtrain = lgb.Dataset(data=x_adv.drop(['C13',  'C1',  'C14', 'D2',                                                             'V87', 'C12', 
 'V156',  'V317',  'card2'],axis=1), label=y_adv,categorical_feature=object_features.to_list())

In [290]:
model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain],
    categorical_feature=object_features.to_list(),
    early_stopping_rounds=90,
    verbose_eval=None
)
model.best_score, model.best_iteration

(defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 1.0)])}),
 161)

In [291]:
pred = model.predict(x_adv.drop(['C13',  'C1',  'C14', 'D2',                                                             'V87', 'C12', 
 'V156',  'V317',  'card2'],axis=1))

In [292]:
pd.cut(
    pred, bins=np.arange(0, 1.01, 0.5)
).value_counts().sort_index()

(0.0, 0.5]    179999
(0.5, 1.0]    100002
dtype: int64

In [293]:
a = list(zip(model.feature_name(), model.feature_importance()))

In [294]:
a.sort(key=lambda x: x[1],reverse=True)
a

[('TransactionAmt', 458),
 ('TransactionDT', 322),
 ('card1', 311),
 ('addr1', 288),
 ('D15', 243),
 ('dist1', 241),
 ('card5', 229),
 ('D4', 169),
 ('D10', 147),
 ('D5', 142),
 ('D3', 140),
 ('D9', 128),
 ('V307', 119),
 ('D1', 92),
 ('D11', 92),
 ('dist2', 84),
 ('V291', 68),
 ('C5', 67),
 ('M6', 54),
 ('D8', 53),
 ('M5', 51),
 ('V314', 51),
 ('M8', 48),
 ('V310', 48),
 ('M9', 40),
 ('V7', 36),
 ('V286', 36),
 ('C2', 34),
 ('V12', 34),
 ('V306', 34),
 ('V130', 33),
 ('C6', 32),
 ('V127', 31),
 ('V308', 31),
 ('V70', 28),
 ('V83', 28),
 ('M4', 27),
 ('M7', 27),
 ('V45', 27),
 ('V221', 25),
 ('C11', 24),
 ('C9', 23),
 ('V86', 23),
 ('M2', 22),
 ('D6', 21),
 ('M3', 20),
 ('V48', 19),
 ('V75', 19),
 ('V285', 17),
 ('V311', 16),
 ('V49', 15),
 ('V96', 15),
 ('V10', 14),
 ('V36', 14),
 ('V39', 14),
 ('V44', 13),
 ('V320', 13),
 ('V4', 12),
 ('V52', 12),
 ('V264', 12),
 ('V309', 12),
 ('V53', 11),
 ('V124', 11),
 ('V294', 11),
 ('V34', 10),
 ('V35', 10),
 ('V99', 10),
 ('V102', 10),
 ('V312